In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import pandas as pd
import os
import random
import csv
import timeit
from sys import platform
from IPython.core.display import HTML
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

/Users/abir.deb/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/abir.deb/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
start = timeit.default_timer()

def cv_score(clf, x, y, score_func=accuracy_score):
    result = 0
    nfold = 5
    for train, test in KFold(y.size, nfold): # split data into train/test groups, 5 times
        clf.fit(x[train], y[train]) # fit
        result += score_func(clf.predict(x[test]), y[test]) # evaluate score function on held-out data
    return result / nfold # average



def replace_STL_with_LA(GAME_df, DRIVE_df, PLAY_df):
    '''
    replaces all instances of 'STL' with 'LA' returns the altered dataframe; this is to stay
    consistent with the NFL Savant modeling approach since 'STL' and 'LA' are both the same Rams franchise/team
    '''
    #change certain columns in GAME_df
    GAME_df['h'] = GAME_df['h'].replace(to_replace = 'STL', value = 'LA')
    GAME_df['v'] = GAME_df['v'].replace(to_replace = 'STL', value = 'LA')
    
    #change certain columns in DRIVE_df
    DRIVE_df['tname'] = DRIVE_df['tname'].replace(to_replace = 'STL', value = 'LA')
    
    #change certain columns in PLAY_df
    PLAY_df['off'] = PLAY_df['off'].replace(to_replace = 'STL', value = 'LA')
    PLAY_df['def'] = PLAY_df['def'].replace(to_replace = 'STL', value = 'LA')
    
    return [GAME_df, DRIVE_df, PLAY_df]


def load_data(dir, lst_files_to_load, path_slash):
    '''
    LOAD DATA from dir that contains data files in csv format; path_slash contains a string used for directories
    #in a path ('/' in Unix based system or '\\' in Windows)
    '''
    files = os.listdir(dir)
    #initialize a dictionary to store csv files
    data_dict = {}
    for fil in files:
        if fil in lst_files_to_load:
            filePath = dir + path_slash + fil
            data_dict[fil[:-4]] = pd.DataFrame.from_csv(filePath)
    return data_dict



def generate_teams_lst(df, year):
    #stores all the team abbreviation (assuming dataframe passed is GAME_df)
    df = df[df.seas == year]
    teams = df.h.dropna().drop_duplicates().sort_values()
    return list(teams)



def check_number_of_teams(df, team_of_interest):
    '''
    debugging check for distinct number of teams (home and away) that faces the team of interest
    when joining drive and game data together and filtering out for year and team of interest
    '''
    home_teams = list(df.h.drop_duplicates())
    away_teams = list(df.v.drop_duplicates())
    all_teams = home_teams + away_teams
    
    #get rid of entries that are the team of interest
    for entry in all_teams:
        if entry == team_of_interest:
            all_teams.remove(entry)
    
    return [len(set(all_teams)), set(all_teams)]



def train_test_qc_check(train, test):
    '''
    check that the labels for the train and test sets are the same and in the same order; this makes sure
    there are no errors when performing linear algebra on the matrices when building the model; returns
    True if all the column labels are in the same and in the same order, False otherwise
    '''
    train_columns = list(train.columns)
    test_columns = list(test.columns)
    
    #make sure both the train and test sets have the same number of columns
    if len(train_columns) != len(test_columns):
        return False
    
    #make sure every column is the same for both the train and test sets; return True if it passes this block
    for i in range(len(train_columns)):
        if train_columns[i] != test_columns[i]:
            return False
    
    return True


def gather_drive_data(GAME_df, DRIVE_df, year, team):
    '''
    This function filters drive data by team and year and returns DRIVE_df with 'gid', 'fpid', and 'plays'
    which are all necessary to gather play by play data in later step
    '''
    columns_of_interest = ['gid', 'fpid', 'plays', 'h', 'v']
    GAME_df_filtered = GAME_df[GAME_df.seas == year]
    DRIVE_df_filtered = DRIVE_df[DRIVE_df.tname == team]
    GAME_DRIVE_joined_df = DRIVE_df_filtered.join(GAME_df_filtered, on = 'gid', how = 'inner')
    
    return GAME_DRIVE_joined_df[columns_of_interest]



def gather_game_ids(df):
    '''
    drive_data_df will be passed here and gameIDs will be gathered and returned in a list
    '''
    game_id_list = list(df.gid.dropna().drop_duplicates())
    
    return game_id_list


def create_train_test_set(df, rand_state):
    '''
    This function creates a train and test set for each game in the filtered drive data (drive_data_df); this ensures
    that each team has a balanced amount of drives in the train and test set for each and every game in a given season
    '''
    #TO DO: iterate through each gameID and filter out those drives and create a separate train/test set for each
    #game so this ensures there's drives for every game in the train/test set, then combine all the dfs and return
    #the train/test df's of drive data
    game_id_lst = gather_game_ids(df)
    train_set_df_lst = []
    test_set_df_lst = []
    
    for gameid in game_id_lst:
        drives_subset_df = df[df.gid == gameid]
        drive_train, drive_test = train_test_split(drives_subset_df, random_state = rand_state)
        train_set_df_lst.append(drive_train)
        test_set_df_lst.append(drive_test)
    
    return [pd.concat(train_set_df_lst, copy = False), pd.concat(test_set_df_lst, copy = False)]

    
def find_plays(first_play_id, num_plays, sub_play_df):
    '''
    return a data frame with all the plays (RUN OR PASS) from a given first_play_id, num_plays and sub_play_df
    (which contains all the plays we need to filter out and return as a data frame)
    '''
    columns_of_interest = [
        'pid', 'off', 'def', 'type', 'qtr', 'min', 'sec'
        , 'ptso', 'ptsd', 'dwn', 'ytg', 'yfog', 'zone'
         , 'sg', 'nh', 'timo', 'timd'
    ]
    
    #calculate the last play's play_id
    last_play_id = first_play_id + num_plays
    
    #gather all the pass/rush plays from sub_play_df using the range of play_ids given for the drive
    all_plays_in_drive = sub_play_df[np.logical_and(np.logical_or(sub_play_df.type == 'PASS', sub_play_df.type == 'RUSH'),
                                            np.logical_and(sub_play_df.pid >= first_play_id, sub_play_df.pid < last_play_id))]
    
    return all_plays_in_drive[columns_of_interest]



def add_under_center_column(df):
    '''
    This function takes in play by play data frame and adds in an under_center column (uc) and returns the data frame
    '''
    df['uc'] = np.int64(df['sg'] == 0)
    
    return df



def gather_play_by_play_data(drive_data_df, PLAY_df):
    '''
    This function returns the play by play data in a data frame from the given drive data and PLAY_df
    '''
    plays_df_lst = [] #holds all the play by play data frames in one list
    game_id = None #will hold the current game_id of interest

    for lab, row in drive_data_df.iterrows():
        #gather necessary variables to extract play by play data
        first_play_id = row[1]
        num_plays = row[2]
        
        #we don't want to keep extracting the same game's data every iteration unless it's a new game_id
        if game_id != row[0]:
            game_id = row[0]
            sub_play_df = PLAY_df.loc[game_id] #partition PLAY_df to play data of interest

        #gather all the plays for this specific drive and store it in plays_df_lst
        all_plays_in_drive = find_plays(first_play_id, num_plays, sub_play_df)
        
        #add data frame of plays to list
        plays_df_lst.append(all_plays_in_drive)
        
    #return all play by play data concatenated and add under center column
    return add_under_center_column(pd.concat(plays_df_lst, copy = False))

    

def create_x_y_matrix(df):
    '''
    This function creates the x and y vector for each data frame with play by play data
    '''
    X = df.drop('type', axis = 1)
    y = df.type.to_frame('play_type')
    
    return [X, y]
    
    
    
def preprocess_data(df):
    '''
    This function will apply preprocessing steps to the train and test set so that they are
    ready for model building
    '''
    features = [
        'off', 'def', 'qtr', 'min', 'sec'
        , 'ptso', 'ptsd', 'dwn', 'ytg', 'yfog', 'zone'
         , 'sg', 'nh', 'uc', 'timo', 'timd'
    ]
    
    #will create X (design matrix) and y (result vector)
    X, y = create_x_y_matrix(df)
    
    #rename column 'type' to 'play_type'
    #X.rename(columns = {'type': 'play_type'}, inplace = True)
    
    #take columns of interest
    X_subset = X[features]
    
    #drop rows that have NaNs in any column
    X_final = X_subset.dropna()
    
    #create a label encoding in y dataframe since 'play_type' is the variable we are trying to predict
    y.play_type = y.play_type.astype('category')
    y['play_type_cat'] = y.play_type.cat.codes
    
    #create dummy variables for each feature that is categorical or an indicator variable
    #(off, def, sg, nh, uc, and zone)
    X_final = pd.get_dummies(X_final, columns = ['off', 'def', 'zone'])
    
    return [X_final, y]



def round_array(array):
    '''return a list of floats that have been rounded from given array'''
    lst_to_return = []
    for entry in array:
        lst_to_return.append(round(entry))
    return lst_to_return



def run_data_wrangling_process(year, team):
    '''
    This function begins the data wrangling/ETL process on the loaded data
    and returns the train and test set data frames ready for model building
    '''
    #gather all the necessary drive data for particular year
    drive_data_df = gather_drive_data(GAME_df, DRIVE_df, year, team)
    
    #create a train and test set of the drive data
    drive_train, drive_test = create_train_test_set(drive_data_df, rand_state = 69)
    
    '''
    #debugging check
    print team
    num_teams_train_set, all_teams_train = check_number_of_teams(drive_train, team)
    num_teams_test_set, all_teams_test = check_number_of_teams(drive_test, team)
    print 'Number of teams in training set', num_teams_train_set
    print all_teams_train
    print 'Number of teams in testing set', num_teams_test_set
    print all_teams_test
    all_teams_train_set = set(all_teams_train)
    all_teams_test_set = set(all_teams_test)
    print 'Team in train or test, but not in both', all_teams_train_set.symmetric_difference(all_teams_test_set)
    print '\n'
    '''
    
    #derive the play by play data from the drive data train and test sets (all plays in each drive stay within each set)
    X_train, y_train = preprocess_data(gather_play_by_play_data(drive_train, PLAY_df))
    X_test, y_test = preprocess_data(gather_play_by_play_data(drive_test, PLAY_df))
    
    return [X_train, y_train, X_test, y_test]



def build_model(X_train, y_train, X_test, y_test, year, team):
    '''
    This function builds the model from a train and test set and gathers model evaluation metrics
    '''
    #fit a logistic regression model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    
    #accuracy score on train data
    train_accuracy_score_no_grid = accuracy_score(clf.predict(X_train), y_train)
    
    #accuracy score on test data
    test_accuracy_score_no_grid = accuracy_score(clf.predict(X_test), y_test)
    
    #5 Fold Cross Validation Accuracy
    five_fold_cv_score_no_grid = cv_score(clf, X_train, y_train)
    
    #the grid of parameters to search over
    Cs = [0.001, 0.1, 1, 10, 100]
    param_grid = {'C' : Cs }
    model = GridSearchCV(LogisticRegression(), param_grid, scoring = 'accuracy', cv = 5)
    model.fit(X_train, y_train)
    
    #best regularization parameter from grid search
    best_reg = model.best_params_['C']
    
    #test set accuracy with the best regularization parameter
    test_accuracy_best_reg = accuracy_score(model.predict(X_test), y_test)
    
    #gather model metrics using the best regularization parameter
    preds = model.predict_proba(X_test)[:,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, preds)

    y_test_rounded = round_array(y_test)
    preds_rounded = round_array(preds)

    #precision (true positives / [true positives + false positives])
    precision = metrics.precision_score(y_test_rounded, preds_rounded)

    #recall (true positives / [true positives + false negatives])
    recall = metrics.recall_score(y_test_rounded, preds_rounded)

    #F1 score is the geometric/harmonic mean of precision and recall; 2 * tp / ( 2 * tp + fp + fn)
    f1_score = metrics.f1_score(y_test_rounded, preds_rounded)
    
    #AUC
    auc = metrics.auc(fpr, tpr)
    
    #list of metrics
    metrics_lst = [
        year,
        team,
        train_accuracy_score_no_grid,
        test_accuracy_score_no_grid,
        five_fold_cv_score_no_grid,
        best_reg,
        test_accuracy_best_reg,
        precision,
        recall,
        f1_score,
        auc
    ]
    
    #rates to make ROC curve
    roc_curve_metrics = [year, fpr, tpr, auc]
    
    #gather model coefficients
    model_coefficients_tup = tuple(model.best_estimator_.coef_[0].tolist())
    
    lst_to_return = [metrics_lst, roc_curve_metrics]
    
    return lst_to_return

In [3]:
#these are the relational tables in csv format to load into pandas data frames
lst_files_to_load = ['DRIVE.csv', 'GAME.csv', 'PLAY.csv']

#make sure directory paths are correct based on running in a Windows/Unix environment
if platform == 'win32':
    #code is running in a windows machine
    dir_files_to_load = os.getcwd() + '\\armchair\\nfl_00-16'
    data_dict = load_data(dir_files_to_load, lst_files_to_load, '\\')
    metrics_csv_file_path = os.getcwd() + '\\armchair_all_data_model_metrics.csv'
else:
    dir_files_to_load = os.getcwd() + '/armchair/nfl_00-16'
    data_dict = load_data(dir_files_to_load, lst_files_to_load, '/')
    metrics_csv_file_path = os.getcwd() + '/armchair_all_data_model_metrics.csv'

In [4]:
#create all the variables for the dataframes in the data_dict
for df in data_dict.keys():
    command = str(df) + '_df' + ' = data_dict["' + str(df) + '"]'
    #print command
    exec(command)
    
'''
let's replace STL with LA in all the based dataframes to stay consistent with our team based logistic regression
approach with NFL Savant data; STL becomes LA in 2016, so we are assuming STL was LA in 2014-2015 as well just to
stay consistent from a modeling perspective (they are both the same Rams franchise/same team, just relocated)
'''
GAME_df, DRIVE_df, PLAY_df = replace_STL_with_LA(GAME_df, DRIVE_df, PLAY_df)

In [5]:
years = [2014, 2015, 2016]
model_metrics_lsts = [] #will hold all the model metrics for each model generated for each team in each season
roc_curve_rates = [] #will hold fpr & tpr to build ROC curve
columns = [
        'year',
        'team',
        'train_accuracy_score_no_grid',
        'test_accuracy_score_no_grid',
        'five_fold_cv_score_no_grid',
        'best_reg',
        'test_accuracy_best_reg',
        'precision',
        'recall',
        'f1_score',
        'auc'
    ]


for year in years:
    #generate a new list of teams for each season (teams may be different in certain seasons)
    teams_lst = generate_teams_lst(GAME_df, year)
    for team in teams_lst:
        X_train, y_train, X_test, y_test = run_data_wrangling_process(year, team)
        
        #QC for train and test set transformations (from drive train/tests to play by play train/test sets)
        result_x = train_test_qc_check(X_train, X_test)
        result_y = train_test_qc_check(y_train, y_test)
        
        if result_x == False:
            print team, year, 'X_train and X_test mismatch'
            
        if result_y == False:
            print team, year, 'y_train and y_test mismatch'
            
        #each year's metrics results will be held in metrics_lst, tpr & fpr are used for ROC Curve
        model_metrics_lst, roc_curve_metrics = build_model(X_train.values, y_train['play_type_cat'].values
                                                     , X_test, y_test['play_type_cat'].values, year, team)
        model_metrics_lsts.append(model_metrics_lst)
    
#convert metrics_lsts into a dataframe
metrics_df = pd.DataFrame(model_metrics_lsts, columns = columns)

In [6]:
metrics_df.head(20)

,year,team,train_accuracy_score_no_grid,test_accuracy_score_no_grid,five_fold_cv_score_no_grid,best_reg,test_accuracy_best_reg,precision,recall,f1_score,auc
0,2014,ARI,0.769452,0.757143,0.762308,0.100,0.757143,0.682927,0.743363,0.711864,0.810397
1,2014,ATL,0.749650,0.733083,0.707692,0.100,0.729323,0.586777,0.763441,0.663551,0.779725
2,2014,BAL,0.734513,0.682119,0.701608,0.100,0.682119,0.656489,0.627737,0.641791,0.768016
3,2014,BUF,0.755650,0.671815,0.700659,0.100,0.691120,0.611111,0.550000,0.578947,0.748491
4,2014,CAR,0.661597,0.651685,0.626026,0.100,0.654494,0.654135,0.530488,0.585859,0.717734
5,2014,CHI,0.732746,0.688462,0.690146,0.100,0.688462,0.588235,0.430108,0.496894,0.725195
6,2014,CIN,0.772666,0.724382,0.748272,0.100,0.703180,0.690141,0.710145,0.700000,0.776462
7,2014,CLE,0.728571,0.692000,0.710000,0.100,0.688000,0.630435,0.763158,0.690476,0.781476
8,2014,DAL,0.861601,0.827692,0.860314,0.100,0.827692,0.809249,0.858896,0.833333,0.872037
9,2014,DEN,0.731469,0.674487,0.714685,0.100,0.677419,0.723214,0.506250,0.595588,0.765055


**write metrics_df to a csv file**

In [7]:
metrics_df.to_csv(metrics_csv_file_path)

In [8]:
stop = timeit.default_timer()
program_time_in_sec = stop - start
program_time_in_min = program_time_in_sec / 60.0
print str(round(program_time_in_sec, 1)), 'sec'
print str(round(program_time_in_min, 2)), 'min'

53.2 sec
0.89 min
